In [1]:
!apt-get install gcc
!apt-get install make
!apt-get install libgmp3-dev
!pip install pysmt
!pysmt-install --check
!pysmt-install --msat

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
gcc set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgmp-dev libgmpxx4ldbl
Suggested packages:
  gmp-doc libgmp10-doc libmpfr-dev
The following NEW packages will be installed:
  libgmp-dev libgmp3-dev libgmpxx4ldbl
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 327 kB of archives.
After this operation, 1,689 kB of additional disk space will be used.
Get:1 http://archive.ubunt

In [2]:
from pysmt.shortcuts import Symbol, And, Not, is_sat
varA = Symbol("A") # Default type is Boolean
varB = Symbol("B")
f = And(varA, Not(varB))
f

/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.7/dist-packages/pysmt/smtlib/parser/parser.py
  tree = Parsing.p_module(s, pxd, full_module_name)


(A & (! B))

In [3]:
is_sat(f)
g = f.substitute({varB: varA})
g

(A & (! A))

In [4]:
is_sat(g)

False

In [21]:
from pysmt.shortcuts import Symbol
from pysmt.typing import INT

h = Symbol("H", INT)

domain = (1 <= h) & (10 >= h) & (10 >= h)

In [23]:
type(domain)
a = list(domain.get_atoms())
a0 = a[1]
a1 = (2 <= h)
d1 = domain.substitute({a0: a1})
d1

(((1 <= H) & (2 <= H)) & (2 <= H))

In [17]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver
from pysmt.typing import INT, BOOL

hello = [Symbol(s, INT) for s in "hello"]
world = [Symbol(s, BOOL) for s in "world"]
letters = set(hello+world)

PysmtTypeError: ignored

In [30]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver, serialize
from pysmt.typing import INT, BOOL
from pysmt.parsing import parse

# Replacement of integer expressions in equation by boolean expressions:
# Before:
#   domain = (1 <= h) & (10 >= h)
# After:
#   domain = A & B
equation = "(1 <= h) & (10 >= h)".lower()
eq_variables = [v for v in equation if v.isalpha()]

int_variables = set(Symbol(s, INT) for s in eq_variables)
# bool_variables = set(Symbol(s + "*", BOOL) for s in eq_variables)

# Equation
# domain = (1 <= h) & (10 >= h)
domain = parse(equation)
# a = list(domain.get_atoms())
# print(int_variables)
# print(bool_variables)
# d = dict(zip(int_variables, bool_variables))
atoms = domain.get_atoms()
bool_variables = set(Symbol(s + "*", BOOL) for s in eq_variables)
len(atoms)
# d = dict(zip(atoms, bool_variables))
# domain.substitute({a0: a1})
# domain.substitute({a0: a1})

((1 <= h) & (h <= 10))


In [33]:
print(list(enumerate(atoms)))

[(0, (1 <= h)), (1, (h <= 10))]


In [14]:
a = [i for i in "ñslaiieosnie>=11" if i.isalpha()]
a

['ñ', 's', 'l', 'a', 'i', 'i', 'e', 'o', 's', 'n', 'i', 'e']

In [ ]:
get_free_variables()

In [37]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver, serialize
from pysmt.typing import INT, BOOL
from pysmt.parsing import parse

# Replacement of integer expressions in equation by boolean expressions:
# Before:
#   domain = (1 <= h) & (10 >= h)
# After:
#   domain = A & B
equation = "(1 <= h) & (10 >= h)".lower()
eq_variables = set(v for v in equation if v.isalpha())
int_variables = set(Symbol(s, INT) for s in eq_variables)

# Equation
# domain = (1 <= h) & (10 >= h)
domain = parse(equation)
# a = list(domain.get_atoms())
# print(int_variables)
# print(bool_variables)
# d = dict(zip(int_variables, bool_variables))
atoms = domain.get_atoms()
sorted_atoms = sorted(atoms)
# for a in atoms:
#  print(a, str(a.get_free_variables()))
# bool_variables = set(Symbol( + "*", BOOL) for (a, i) in enumerate(atoms))
# len(atoms)
# d = dict(zip(atoms, bool_variables))
# domain.substitute({a0: a1})
# domain.substitute({a0: a1})

(1 <= h) frozenset({h})
(h <= 10) frozenset({h})


In [41]:
from pysmt.shortcuts import Symbol, LE, GE, Int, And, Equals, Plus, Solver, serialize
from pysmt.typing import INT, BOOL
from pysmt.parsing import parse

# Replacement of integer expressions in equation by boolean expressions:
# Before:
#   equation = (1 <= h) & (10 >= h)
# After:
#   equation = A & B
equation = "(1 <= h) & (10 >= h)".lower()
eq_variables = set(v for v in equation if v.isalpha())
int_variables = set(Symbol(s, INT) for s in eq_variables)

# Equation
# formula = (1 <= h) & (10 >= h)
formula = parse(equation)

# Replace atoms by symbolic variables.
# E.g.: 
#  - "(1 <= h)" by "a0"
#  - "(10 >= h)" by "a1"

atoms = formula.get_atoms()
sorted_atoms = sorted(atoms)
bool_variables = set(Symbol("a" + str(i), BOOL) for (i, a) in enumerate(sorted_atoms))
form_2_abstr = dict(zip(bool_variables, atoms))
# formula           = (1 <= h) & (h <= 10)
# abstract_formula  = a0 & a1
abstract_formula = formula.substitute(form_2_abstr)

# Call to SETTA method

abstr_2_form = dict(zip(atoms, bool_variables))
fp = abstract_formula.substitute(abstr_2_form)
# abstract_formula  = a0 & a1
# fp           = (1 <= h) & (h <= 10)

print(formula)
print(fp)
print(abstract_formula)

((1 <= h) & (h <= 10))
(a0 & a1)
((1 <= h) & (h <= 10))
